<a href="https://colab.research.google.com/github/Aarthi0705/Aarthi0705/blob/main/do%20in%20other%20colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install -q transformers sentence-transformers qdrant-client networkx rank-bm25

for umls

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
book_path = "/content/drive/My Drive/rag/rag1/anatomy.pdf"

In [3]:
!pip install -q pymupdf

read book


In [4]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = "\n".join([page.get_text() for page in doc])
    return text

# Set your book path (modify if needed)
book_path = "/content/drive/My Drive/rag/rag1/anatomy.pdf"

# Extract text
book_text = extract_text_from_pdf(book_path)

print("✅ Successfully extracted text! First 500 characters:")
print(book_text[:500])  # Preview


✅ Successfully extracted text! First 500 characters:


USMLE Step 1 
Dr. Jack Wilson, PhD 
National Instructor 
•aOJEII IOMA l EOUCATI O iif 
v 1.2 

Dr. Jack Wilson, PhD 
Professor of Anatomy and Neurobiology 
University of Tennessee Health Science Center 
Memphis, TN 
Steven R. Daugherty, PhD 
Director, Faculty and Curriculum at Becker Professional Education 
Chicago, IL 
The United States Medical Licensing Examination® (USMLE®) is a joint program of the Federation 
of State Medical Boards (FSMB) and National Board of Medical Examiners® (NBME<!>


quick umls we will use but it didnt workout so


In [5]:
!pip install -q quickumls spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 54.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


Preprocess data

In [6]:
import re
import unicodedata

def normalize_text(text):
    # Convert Unicode characters to normal form
    text = unicodedata.normalize("NFKC", text)

    # Remove extra spaces and line breaks
    text = re.sub(r"\s+", " ", text)

    # Remove non-ASCII characters (optional)
    text = re.sub(r"[^\x00-\x7F]+", " ", text)

    # Normalize punctuation spacing
    text = re.sub(r"([.,!?()])", r" \1 ", text)

    return text.strip()

# Apply normalization
normalized_text = normalize_text(book_text)

print("✅ Text normalized! First 500 characters:")
print(normalized_text[:2000])  # Preview


✅ Text normalized! First 500 characters:
USMLE Step 1 Dr .  Jack Wilson ,  PhD National Instructor  aOJEII IOMA l EOUCATI O iif v 1 . 2 Dr .  Jack Wilson ,  PhD Professor of Anatomy and Neurobiology University of Tennessee Health Science Center Memphis ,  TN Steven R .  Daugherty ,  PhD Director ,  Faculty and Curriculum at Becker Professional Education Chicago ,  IL The United States Medical Licensing Examination   ( USMLE  )  is a joint program of the Federation of State Medical Boards  ( FSMB )  and National Board of Medical Examiners   ( NBME< ! > )  .  United States Medical Licensing Examination ,  USMLE ,  National Board of Medical Examiners ,  and NBME are registered trademarks of the National Board of Medical Examiners .  The National Board of Medical Examiners does not sponsor ,  endorse ,  or support Becker Professional Education in any manner .    2013 by De . Vry/Becker Educational Development Corp .  AU rights rese . rved .  No part of this work may be reproduced ,  transl

Hierarchy chunking

In [7]:
import re

def hierarchical_chunking(text):
    """
    Splits text into hierarchical levels:
    - High-level: Based on section headers (e.g., CHAPTER, SECTION)
    - Mid-level: Paragraph-based
    - Low-level: Sentence-based fine-grained chunks
    """
    # Step 1: Split into high-level chunks (Assumes chapters start with "CHAPTER" or "SECTION")
    high_level_chunks = re.split(r"(?=\b(?:CHAPTER|SECTION)\s+\d+)", text, flags=re.IGNORECASE)

    structured_chunks = []

    for high_chunk in high_level_chunks:
        # Step 2: Split each high-level chunk into mid-level chunks (paragraphs)
        mid_level_chunks = high_chunk.split("\n\n")  # Assuming paragraphs are separated by double newlines

        for mid_chunk in mid_level_chunks:
            # Step 3: Split into low-level chunks (sentences)
            low_level_chunks = re.split(r"(?<=[.!?])\s+", mid_chunk)  # Sentence tokenization

            structured_chunks.append(low_level_chunks)  # Store hierarchical chunks

    return structured_chunks

# Apply hierarchical chunking
hierarchical_chunks = hierarchical_chunking(normalized_text)

# Print preview of first hierarchical chunk
print("✅ Hierarchical Chunking Completed! Sample Output:")
print(hierarchical_chunks[200:300])  # First two structured chunks
print(len(hierarchical_chunks))

✅ Hierarchical Chunking Completed! Sample Output:
[['Chapter 7-17 '], ['Chapter 7   Thorax Septation of the Heart Tube The original heart tube develops as a common clhamber except for the sinus venosus that originally develops right and left .', 'The heart tube undergoes major septation events to divide it into a right and left heart .', 'The atrial ,  ventricular ,  and truncal septations occur concurrently .', 'Most of the major partitioning of t he heart occurs in the fourth and fifth weeks and is usually completed by the eighth week .', '6 .', '1 Septation of the Atria A Membranous portion of interventricular septum 0 SE:ptum pnmum Ostium primum 0~- .', '.', '.', '.', '.', '::;;~-- Endocardial cushion  ( neural crest )  Intraventricular foramen Valve of foramen ovale  ( dosed ovale )  Muscluar portion of interventricular septum  .', '& Figure 7- 6 .', '1 A Atrial Septation   OeVry/Becker Educational Development Corp .', 'All rights reserved .', 'Anatomy '], ['Chapter 7- 18 '], ['Ch

In [14]:
def clean_chunk(chunk):
    """Removes unwanted symbols, numbers, and very short chunks."""
    chunk = chunk.strip()

    # Remove if the chunk is just punctuation or numbers
    if re.match(r"^[\d\s\W]+$", chunk):  # Matches chunks with only numbers or special chars
        return None

    # Remove very short chunks (less than 5 words)
    if len(chunk.split()) < 5:
        return None

    return chunk

def refined_agentic_chunking(hierarchical_chunks, alpha=1.0, beta=1.0):
    """Applies agentic chunking while filtering unwanted chunks."""
    refined_chunks = []

    for level in hierarchical_chunks:
        for chunk in level:
            # Compute importance score
            importance_score = (
                alpha * compute_token_density(chunk) +
                beta * compute_semantic_entropy(chunk)
            )

            # Apply cleaning
            cleaned_chunk = clean_chunk(chunk)
            if cleaned_chunk:
                # If important, further split by sentence
                if importance_score > 1.5:
                    refined_chunks.extend(clean_chunk(sentence) for sentence in chunk.split(". ") if clean_chunk(sentence))
                else:
                    refined_chunks.append(cleaned_chunk)  # Keep as is

    return [c for c in refined_chunks if c]  # Remove None values

# Apply refined chunking
cleaned_chunks = refined_agentic_chunking(hierarchical_chunks)

print(f"✅ Cleaned & Reduced Chunk Count: {len(cleaned_chunks)}")
print("🔹 Sample Cleaned Chunks:", cleaned_chunks[:5])
print(len(cleaned_chunks))

✅ Cleaned & Reduced Chunk Count: 5593
🔹 Sample Cleaned Chunks: ['USMLE Step 1 Dr .', 'Jack Wilson ,  PhD National Instructor  aOJEII IOMA l EOUCATI O iif v 1 .', 'Jack Wilson ,  PhD Professor of Anatomy and Neurobiology University of Tennessee Health Science Center Memphis ,  TN Steven R .', 'Daugherty ,  PhD Director ,  Faculty and Curriculum at Becker Professional Education Chicago ,  IL The United States Medical Licensing Examination   ( USMLE  )  is a joint program of the Federation of State Medical Boards  ( FSMB )  and National Board of Medical Examiners   ( NBME< !', 'United States Medical Licensing Examination ,  USMLE ,  National Board of Medical Examiners ,  and NBME are registered trademarks of the National Board of Medical Examiners .']
5593


Agentic chunking

In [9]:
!pip install -q sentence-transformers

Implement Agentic Chunking

In [16]:
from sentence_transformers import SentenceTransformer

# Load sentence embedding models
semantic_model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
long_context_model = SentenceTransformer("sentence-transformers/all-distilroberta-v1")

print("✅ Models Loaded Successfully!")

✅ Models Loaded Successfully!


In [ ]:
import numpy as np

def compute_batch_embeddings(chunks, batch_size=32, w_s=0.6, w_c=0.4):
    """
    Computes hybrid embeddings in batches for faster processing.
    """
    embeddings = []

    for i in range(0, len(chunks), batch_size):
        batch = chunks[i : i + batch_size]

        # Compute embeddings in batch
        v_s = semantic_model.encode(batch, convert_to_numpy=True, batch_size=batch_size)  # Semantic
        v_c = long_context_model.encode(batch, convert_to_numpy=True, batch_size=batch_size)  # Long-Context

        # Weighted fusion & normalization
        v_final = (w_s * v_s) + (w_c * v_c)
        v_final = v_final / np.linalg.norm(v_final, axis=1, keepdims=True)

        embeddings.extend(v_final)

    return embeddings

# Compute embeddings efficiently
chunk_embeddings = compute_batch_embeddings(agentic_chunks, batch_size=32)

print(f"✅ Computed embeddings for {len(chunk_embeddings)} chunks!")
print(f"🔹 Sample Embedding Shape: {chunk_embeddings[0].shape}")


In [ ]:
import re
import numpy as np
import networkx as nx
from collections import Counter
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi

# Define helper functions
def compute_token_density(text):
    tokens = text.split()
    freq = Counter(tokens)
    return sum(freq.values()) / len(tokens)

def compute_semantic_entropy(text):
    tokens = text.split()
    freq = Counter(tokens)
    probs = np.array([freq[t] / len(tokens) for t in tokens])
    return -np.sum(probs * np.log(probs + 1e-10))

def compute_umls_entity_score(text, umls_terms):
    return sum(1 for term in umls_terms if term in text)

def compute_graph_centrality(text, graph):
    entities = text.split()
    return sum(nx.pagerank(graph).get(e, 0) for e in entities)

# Adaptive Chunking Function
def agentic_chunking(text, umls_terms, graph, alpha=1, beta=1, gamma=1, delta=1):
    sentences = re.split(r'(?<=[.!?]) +', text)
    chunks = []
    temp_chunk = []

    for sentence in sentences:
        score = (
            alpha * compute_token_density(sentence) +
            beta * compute_semantic_entropy(sentence) +
            gamma * compute_umls_entity_score(sentence, umls_terms) +
            delta * compute_graph_centrality(sentence, graph)
        )
        if score > 1.5:  # Dynamic threshold
            chunks.append(" ".join(temp_chunk))
            temp_chunk = []
        temp_chunk.append(sentence)

    if temp_chunk:
        chunks.append(" ".join(temp_chunk))

    return chunks


Context-Aware Hybrid Embeddings

In [ ]:
# Load Embedding Models
semantic_model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
long_context_model = SentenceTransformer("sentence-transformers/all-distilroberta-v1")

def compute_hybrid_embeddings(text):
    v_s = semantic_model.encode(text)
    v_c = long_context_model.encode(text)
    return 0.6 * v_s + 0.4 * v_c  # Weighted fusion


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

 Store & Retrieve from Qdrant

In [ ]:
# !pip install -q transformers sentence-transformers qdrant-client networkx rank-bm25 pydrive pypdf scispacy spacy pandas
# !pip install -q https://s3.amazonaws.com/biomednlp/umls.json.gz  # UMLS Entity Recognition
# !python -m spacy download en_core_sci_sm  # SciSpacy Model


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.7/298.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.3/243.3 kB 16.4 MB/s eta 0:00:00
  ERROR: HTTP error 404 while getting https://s3.amazonaws.com/biomednlp/umls.json.gz
ERROR: Could not install requirement https://s3.amazonaws.com/biomednlp/umls.json.gz because of HTTP error 404 Client Error: Not Found for url: https://s3.amazonaws.com/biomednlp/umls.json.gz for URL https://s3.amazonaws.com/biomednlp/umls.json.gz


In [ ]:
# from google.colab import drive
# import os

# # Mount Google Drive
# drive.mount('/content/drive')

# # Define the path to your book file in Google Drive
# BOOK_PATH = "/content/drive/MyDrive/rag/rag1/anatomy.pdf"  # Change filename if it's a PDF

# # Check if the file exists
# if not os.path.exists(BOOK_PATH):
#     raise FileNotFoundError(f"Book not found at {BOOK_PATH}")

# # Load the book text
# with open(BOOK_PATH, "r", encoding="utf-8") as file:
#     book_text = file.read()

# print("✅ Book loaded successfully!")
# print(book_text[:500])  # Print first 500 characters to verify


In [ ]:
# !pip install -q pdfplumber
# import pdfplumber

# BOOK_PATH = "/content/drive/MyDrive/book.pdf"

# with pdfplumber.open(BOOK_PATH) as pdf:
#     book_text = "\n".join([page.extract_text() for page in pdf.pages if page.extract_text()])

# print("✅ PDF extracted successfully!")


In [ ]:
from qdrant_client import QdrantClient, models

# Initialize Qdrant
client = QdrantClient(":memory:")

# Check if the collection exists before recreating
if client.collection_exists("medical_rag"):
    client.delete_collection("medical_rag")

client.create_collection(
    collection_name="medical_rag",
    vectors_config=models.VectorParams(size=768, distance=models.Distance.COSINE)
)

# ✅ Define Medical Text
text = """
Lung cancer is one of the leading causes of cancer-related deaths. Symptoms include persistent cough, chest pain, and weight loss.
Risk factors involve smoking, exposure to radon gas, and genetic predisposition. Early diagnosis improves survival rates.
Treatments include surgery, chemotherapy, and immunotherapy.
"""

# ✅ Define Sample UMLS Medical Terms & Knowledge Graph
umls_terms = {"lung cancer", "cough", "chemotherapy", "immunotherapy"}
graph = nx.Graph()
graph.add_edges_from([("lung cancer", "chemotherapy"), ("lung cancer", "immunotherapy")])

# ✅ Store Chunks in Qdrant
for chunk in agentic_chunking(text, umls_terms, graph):
    embedding = compute_hybrid_embeddings(chunk)
    client.upsert(
        collection_name="medical_rag",
        points=[models.PointStruct(id=np.random.randint(1e6), vector=embedding.tolist(), payload={"text": chunk})]
    )

# ✅ Query the System
query = "What are the symptoms of lung cancer?"
query_embedding = compute_hybrid_embeddings(query)
search_results = client.search("medical_rag", query_vector=query_embedding.tolist(), limit=3)

# ✅ Print Retrieved Chunks
for result in search_results:
    print("🔹 Retrieved Chunk:", result.payload["text"])


🔹 Retrieved Chunk: Symptoms include persistent cough, chest pain, and weight loss.
Risk factors involve smoking, exposure to radon gas, and genetic predisposition.
🔹 Retrieved Chunk: 
Lung cancer is one of the leading causes of cancer-related deaths.
🔹 Retrieved Chunk: Early diagnosis improves survival rates.
Treatments include surgery, chemotherapy, and immunotherapy.



<ipython-input-5-2c2ec73a7472>:38: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_results = client.search("medical_rag", query_vector=query_embedding.tolist(), limit=3)
